In [ ]:
import numpy as np
import copy

In [ ]:
# Day 21
with open("input21", "r") as fp:
    lines = fp.readlines()
foods = []
for line in lines:
    parts = line.split("(")
    food = [parts[0].strip().split(" ")]
    if len(parts) > 1:
        food.append([x.strip() for x in parts[1].strip()[8:-1].strip().split(",")])
    else:
        food.append([])
    foods.append(food)
allergies = {}
for food in foods:
    for aller in food[1]:
        if aller not in allergies:
            allergies[aller] = set(food[0])
        else:
            allergies[aller] = set(food[0]).intersection(allergies[aller])
all_ings = [y for x in foods for y in x[0]]
aller_ings = set([y for x in allergies.values() for y in x])
no_allergies = set(all_ings).difference(aller_ings)
print(len([x for x in all_ings if x in no_allergies]))
final_map = {}
while True:
    if (len(final_map) == len(allergies)):
        break
    for aller, ings in allergies.items():
        if len(ings) == 1:
            final_map[aller] = ings
            for k in allergies:
                allergies[k] = allergies[k].difference(ings)
keys = list(final_map.keys())
keys.sort()
print(",".join([list(final_map[k])[0] for k in keys]))

In [ ]:
# Day 20
with open("input20", "r") as fp:
    lines = fp.readlines()
tiles = {}
lines = [lines[i:i+11] for i in range(0,len(lines), 12)]
for line in lines:
    tile_nr = int(line[0].strip()[5:-1])
    tile = []
    for l in line[1:]:
        tile.append([c=="#" for c in l.strip()])
    tiles[tile_nr] = np.array(tile).astype(int)
borders = {}
for k, tile in tiles.items():
    # save [r,ri],[u,ui],[l,li],[d,di]
    borders[k] = [
        [
            int("".join(tile[:,-1].astype(str)),2),
            int("".join(tile[::-1,-1].astype(str)),2)
        ],
        [
            int("".join(tile[0,:].astype(str)),2),
            int("".join(tile[0,::-1].astype(str)),2)
        ],
        [
            int("".join(tile[:,0].astype(str)),2),
            int("".join(tile[::-1,0].astype(str)),2)
        ],
        [
            int("".join(tile[-1,:].astype(str)),2),
            int("".join(tile[-1,::-1].astype(str)),2)
        ]
    ]
all_borders = np.array(list(borders.values()))

neighbors = {}
tile_orientation = "00"
answer = 1
for k, tile in tiles.items():
    # find neighbors
    sumNone = 0
    for b in borders[k]:
        for b1 in b:
            if np.sum(all_borders == b1) == 1:
                sumNone += 1
    if sumNone == 4:
        answer *= k
print(answer)
# Skipped stiching for now :(

In [ ]:
# Day 19
# Maybe overkill using a full feature grammer checker, but payed off in part 2 :)
# Still probably slower than a simple parse
from lark import Lark
with open("input19", "r") as fp:
    lines = fp.readlines()

def parse_it(v2=False):
    rulez = {}
    data = []
    has_empty = False
    for line in lines:
        if line.strip() == "":
            has_empty = True
            continue
        if has_empty:
            data.append(line.strip())
        else:
            rule_nb = int(line.split(":")[0])
            if rule_nb == 8 and v2:
                rule = [[42], [42, 8]]
            elif rule_nb == 11 and v2:
                rule = [[42, 31], [42, 11, 31]]
            else:
                rule = [r.strip() for r in line.split(":")[1].split("|")]
                if '"' in rule[0]:
                    rule = rule[0].replace('"','')
                else:
                    rule = [[int(r2.strip()) for r2 in r.split(" ")] for r in rule]
            rulez[rule_nb] = rule
    grammer = ""
    for i, rule in rulez.items():
        name = f"rule{i}"
        if i == 0:
            name = "start"
        if isinstance(rule, str):
            grammer += f'{name}: "{rule}"\n'
        else:
            grammer += f"{name}: " + " | ".join([" ".join([f"rule{x}" for x in y]) for y in rule]) + "\n"
    l = Lark(grammer)
    correct = 0
    for dat in data:
        try:
            l.parse(dat)
            correct += 1
        except:
            pass
    return correct

print(parse_it(False))
print(parse_it(True))

In [ ]:
# Day 18
import pyparsing
with open("input18", "r") as fp:
    lines = fp.readlines()

content = pyparsing.Word(pyparsing.nums) | "+" | "*"
parser = pyparsing.nestedExpr("(",")",content=content)

def calculate(line):
    num = int(line[0])
    next_op = None
    for part in line[1:]:
        if part == "+":
            next_op = "+"
            continue
        if part == "*":
            next_op = "*"
            continue
        if next_op == "+":
            num += int(part)
        if next_op == "*":
            num *= int(part)
    return num

def calc(line):
    for part in [[i,r] for i, r in enumerate(line) if isinstance(r, list)]:
        line[part[0]] =calc(part[1])
    # now all should be flat, so calculate
    return calculate(line)

sum_ = 0
for line in lines:
    sum_ += calc(parser.parseString(f"({line})").asList())

print(sum_)
# new calculate
def calculate(line):
    plus = np.where(np.array(line) == "+")[0]
    to_rm = []
    for i in plus:
        # by assigning i+1, a + b + c can also be solved :)
        line[i+1] = int(line[i-1]) + int(line[i+1])
        to_rm.append(i-1)
        to_rm.append(i)
    to_rm.sort()
    for i in reversed(to_rm):
        del line[i]
    for i in range(len(line)):
        try:
            line[i] = int(line[i])
        except:
            line[i] = 1
    return np.multiply.reduce(line)
sum_ = 0
for line in lines:
    sum_ += calc(parser.parseString(f"({line})").asList())

print(sum_)

In [ ]:
# Day 17
with open("input17", "r") as fp:
    lines = fp.readlines()
offset = 12
grid = np.zeros([offset*2]*3, bool)
for y, line in enumerate(lines):
    for x, char in enumerate(line):
        grid[offset,offset+y-1,offset+x-1] = char=="#"

for i in range(6):
    mins = [np.min(a)-1 for a in np.where(grid)]
    maxs = [np.max(a)+1 for a in np.where(grid)]
    new_grid = np.zeros_like(grid)
    for z in range(mins[0], maxs[0]+1):
        for y in range(mins[1], maxs[1]+1):
            for x in range(mins[2], maxs[2]+1):
                if grid[z,y,x]:
                    if np.sum(grid[z-1:z+2,y-1:y+2,x-1:x+2])-1 in [2,3]:
                        new_grid[z,y,x] = True
                else:
                    if np.sum(grid[z-1:z+2,y-1:y+2,x-1:x+2]) in [3]:
                        new_grid[z,y,x] = True
    grid = new_grid
print(np.sum(grid))
offset = 12
grid = np.zeros([offset*2]*4, bool)
for y, line in enumerate(lines):
    for x, char in enumerate(line):
        grid[offset,offset+y-1,offset+x-1,offset] = char=="#"

for i in range(6):
    mins = [np.min(a)-1 for a in np.where(grid)]
    maxs = [np.max(a)+1 for a in np.where(grid)]
    new_grid = np.zeros_like(grid)
    for z in range(mins[0], maxs[0]+1):
        for y in range(mins[1], maxs[1]+1):
            for x in range(mins[2], maxs[2]+1):
                for w in range(mins[3], maxs[3]+1):
                    if grid[z,y,x,w]:
                        if np.sum(grid[z-1:z+2,y-1:y+2,x-1:x+2,w-1:w+2])-1 in [2,3]:
                            new_grid[z,y,x,w] = True
                    else:
                        if np.sum(grid[z-1:z+2,y-1:y+2,x-1:x+2,w-1:w+2]) in [3]:
                            new_grid[z,y,x,w] = True
    grid = new_grid
print(np.sum(grid))

In [ ]:
# Day 16
with open("input16", "r") as fp:
    lines = fp.readlines()
rules = {}
yours = []
others = []
for i, line in enumerate(lines):
    if line.strip() == "":
        break
    name = line.split(":")[0].strip()
    rulez = [[int(k) for k in r.split("-")] for r in line.split(":")[1].strip().split("or")]
    rules[name] = rulez
yours = [int(k) for k in lines[i+2].split(",")]
for line in lines[i+5:]:
    others.append([int(k) for k in line.split(",")])
error = 0
correct = []
for other in others:
    keep = True
    for num in other:
        any_correct = False
        for rule in rules.values():
            if rule[0][0] <= num <= rule[0][1] or rule[1][0] <= num <= rule[1][1]:
                any_correct = True
        if not any_correct:
            error += num
            keep = False
            break
    if keep:
        correct.append(other)
print(error)
possibilities = [list(rules.keys()) for k in rules.keys()]
for ticket in correct:
    for i, num in enumerate(ticket):
        for name, rule in rules.items():
            if not(rule[0][0] <= num <= rule[0][1] or rule[1][0] <= num <= rule[1][1]):
                possibilities[i].remove(name)
                final = {}
while True:
    for i, pos in enumerate(possibilities):
        if len(pos) == 1:
            name = pos[0]
            final[name] = i
            for pos2 in possibilities:
                if pos != pos2:
                    try:
                        pos2.remove(name)
                    except:
                        pass
    if all([len(p) == 1 for p in possibilities]):
        break
    answer = 1
for k, v in final.items():
    if k[:len("departure")] == "departure":
        answer *= yours[v]
print(answer)

In [ ]:
# Day 15
numbers = [0,3,6]

def calc(numbers, time):
    last = numbers[-1]
    turn = len(numbers)
    numbers = {n:i for i, n in enumerate(numbers[:-1])}
    for i in range(len(numbers)-1,time-2):
        if last in numbers:
            info = numbers[last]
            numbers[last] = turn - 1
            last = (turn - 1 - info)
        else:
            numbers[last] = turn - 1
            last = 0
        turn += 1
    return last    
    
print(calc(numbers, 2020))
# takes some time, there might be a smarter approach?
print(calc(numbers, 30000000))

In [ ]:
# Day 14
with open("input14", "r") as fp:
    lines = fp.readlines()
mem = {}
current_mask = "X"*36
def override(mask, value):
    # simple iterative
    value = f"{value:036b}"
    result = ""
    for i in range(len(mask)):
        if mask[i] == "X":
            result += value[i]
        else:
            result += mask[i]
    return int(result, 2)
for line in lines:
    inputs = line.split("=")
    if inputs[0].strip() == "mask":
        current_mask = inputs[1].strip()
        continue
    if inputs[0][:3] == "mem":
        mem[inputs[0][4:-1]] = override(current_mask, int(inputs[1].strip()))
print(np.sum(list(mem.values())))

def get_mem_addresses(mask, value):
    # simple iterative
    value = f"{value:036b}"
    result = ""
    x_pos = []
    for i in range(len(mask)):
        if mask[i] == "0":
            result += value[i]
        elif mask[i] == "1":
            result += "1"
        else:
            result += "X"
            x_pos.append(i)
    results = []
    for i in range(2**len(x_pos)):
        index = f"{i:b}".zfill(len(x_pos))
        res = result.split("X")
        res_str = ""
        for ind, ind2 in enumerate(index):
            res_str += res[ind]
            res_str += ind2
        res_str += res[-1]
        results.append(res_str)
    return results
mem = {}
current_mask = "0"*36
for line in lines:
    inputs = line.split("=")
    if inputs[0].strip() == "mask":
        current_mask = inputs[1].strip()
        continue
    if inputs[0][:3] == "mem":
        for mem_add in get_mem_addresses(current_mask, int(inputs[0].strip()[4:-1])):
            mem[mem_add] = int(inputs[1].strip())
print(np.sum(list(mem.values())))

In [ ]:
# Day 13
with open("input13", "r") as fp:
    time = int(fp.readline())
    busses = [int(b) for b in fp.readline().split(",") if b != "x"]
    smallest = np.inf
for bus in busses:
    away = bus - (time % bus)
    if (away < smallest):
        smallest = away
        result = [bus, away]
print(result[0]*result[1])
with open("input13", "r") as fp:
    fp.readline()
    busses = {int(b):i for i, b in enumerate(fp.readline().split(",")) if b != "x"}

# To solve with a system of modulars, 
# (x + index) % busid = 0 ==> x = -index  | mod busid  this must hold for all busses
# searching for it reveals: https://en.wikipedia.org/wiki/Chinese_remainder_theorem

N = np.multiply.reduce([k for k in busses.keys()])
yi = [N//k for k in busses.keys()]
zi = [pow(int(yi[i]), -1, k) for i,k in enumerate(busses.keys())]
print(np.sum([-busses[k]*yi[i]*zi[i] for i, k in enumerate(busses.keys())]) %N)

In [ ]:
# Day 12
with open("input12", "r") as fp:
    lines = fp.readlines()
instructions = []
for line in lines:
    instructions.append([line[0], int(line[1:])])
DIRECTIONS = np.array([(1,0), (0,1), (-1,0), (0,-1)])
DIRS_STRING = "ENWS"
facing = 0
pos = np.array([0,0])
for inst in instructions:
    if inst[0] in DIRS_STRING:
        pos += DIRECTIONS[DIRS_STRING.find(inst[0])]*inst[1]
    if inst[0] == "L":
        facing += inst[1]//90
        facing %= 4
    if inst[0] == "R":
        facing -= inst[1]//90
        facing %= 4
    if inst[0] == "F":
        pos += DIRECTIONS[facing]*inst[1]
print(np.sum(np.abs(pos)))

def rotateCW(point):
    return np.array([point[1], -point[0]])

def rotateCCW(point):
    return np.array([-point[1], point[0]])


pos_ship = np.array([0,0])
pos_waypoint = np.array([10,1])
for inst in instructions:
    if inst[0] in DIRS_STRING:
        pos_waypoint += DIRECTIONS[DIRS_STRING.find(inst[0])]*inst[1]
    if inst[0] == "L":
        for i in range(inst[1]//90):
            pos_waypoint = rotateCCW(pos_waypoint)
    if inst[0] == "R":
        for i in range(inst[1]//90):
            pos_waypoint = rotateCW(pos_waypoint)
    if inst[0] == "F":
        pos_ship += pos_waypoint*inst[1]
print(np.sum(np.abs(pos_ship)))

In [ ]:
# Day 11
# This is most likely not the most performant way to do it, to much array copying...
with open("input11", "r") as fp:
    lines = fp.readlines()

field = []
for y, line in enumerate(lines):
    field.append([])
    for x, char in enumerate(line.strip()):
        field[y].append(".L".find(char))
field = np.array(field)

def get_part(x,y, kind="v1"):
    if kind == "v1":
        return field[max(y-1,0):y+2,max(x-1,0):x+2]
    else:
        dirs = [[-1,0],[1,0],[0,-1],[0,1],[1,1],[1,-1],[-1,1],[-1,-1]]
        result = np.zeros(9)
        result[8] = field[y,x]
        for ind, direction in enumerate(dirs):
            for i in range(1,10000):
                x2 = x+direction[0]*i
                y2 = y+direction[1]*i
                if x2 < 0 or y2 < 0 or x2 >= field.shape[1] or y2 >= field.shape[0]:
                    result[ind] = 0
                    break
                f = field[y2, x2]
                if f > 0:
                    result[ind] = f
                    break
        return np.array(result)


def born(x,y, kind="v1"):
    return np.max(get_part(x, y, kind)) < 2

def die(x,y, kind="v1"):
    part = get_part(x, y, kind)
    if kind == "v1":
        return np.sum(part==2) >= 5
    else:
        return np.sum(part==2) >= 6

def iteration(kind="v1"):
    global field
    new_field = field.copy()
    for y in range(field.shape[0]):
        for x in range(field.shape[1]):
            if field[y,x] == 0:
                continue
            if born(x,y,kind):
                new_field[y,x] = 2
            if die(x,y,kind):
                new_field[y,x] = 1
    field = new_field

original_field = field.copy()
last_field = field.copy()
for i in range(500):
    iteration("v1")
    if np.all(last_field == field):
        print(np.sum(field == 2))
        break
    last_field = field.copy()

field = original_field
last_field = field.copy()
for i in range(500):
    iteration("v2")
    if np.all(last_field == field):
        print(np.sum(field == 2))
        break
    last_field = field.copy()



In [ ]:
# Day 10
with open("input10", "r") as fp:
    lines = fp.readlines()
numbers = []
for line in lines:
    numbers.append(int(line))
numbers.sort()
numbers.append(numbers[-1]+3)
numbers.insert(0, 0)
diffs = np.diff(numbers)
print(len(diffs[diffs==1]) * len(diffs[diffs==3]))
# diff only consists of 1s and 3s. Every 3 "resets" the sequence (every sequence has to go through here)
# the number of 1s between 3s indicates how many different permutation between those two 3s are possible
# (e.g.) 3ABC3 -> 4 ways 3ABC3, 3AB3, 3AC3, 3BC3 (not that we already need one step to get to 3, so we basically can skip one 1)
num_1s = [len(n) -1 for n in np.split([3] + diffs.tolist(), np.where(diffs==3)[0]+1) if len(n) > 1]
mult = [0,1,2,4,7]
multipliers = [mult[i] for i in num_1s]
print(np.multiply.reduce(multipliers))

In [ ]:
# Day 9
with open("input9", "r") as fp:
    lines = fp.readlines()
numbers = []
for line in lines:
    numbers.append(int(line))

def check(number, number_list):
    for n1 in number_list:
        for n2 in number_list:
            if n1+n2 == number:
                return True
    return False

for i in range(25,len(numbers)):
    if not check(numbers[i], numbers[i-25:i]):
        print(numbers[i])
        invalid = numbers[i]
        break
# brute force :(
start = 0
end = 1
while True:
    sum_ = np.sum(numbers[start:end]) 
    if sum_ == invalid:
        print(np.max(numbers[start:end])+np.min(numbers[start:end]))
        break
    if sum_ > invalid:
        start += 1
        end = start + 1
    else:
        end += 1

In [ ]:
# Day 8
with open("input8", "r") as fp:
    lines = fp.readlines()

instructions = []
for line in lines:
    op = line.split()[0]
    num = int(line.split()[1])
    instructions.append([op, num])
acc = 0
visited = []
current = 0
while True:
    if current in visited:
        break
    visited.append(current)
    op = instructions[current]
    if op[0] == "acc":
        acc += op[1]
        current += 1
        continue
    if op[0] == "nop":
        current += 1
        continue
    if op[0] == "jmp":
        current += op[1]
        continue
print(acc)
# Brute force it
# We already know visited
import copy
wrong_visited = copy.copy(visited)
mod = -1
while True:
    instructions_new = copy.copy(instructions)
    # change next instruction from back (skipping acc)
    while True:
        if instructions[wrong_visited[mod]][0] == "nop":
            instructions_new[wrong_visited[mod]][0] = "jmp"
            break
        if instructions[wrong_visited[mod]][0] == "jmp":
            instructions_new[wrong_visited[mod]][0] = "nop"
            break
        mod -= 1
    mod -= 1
    # And test this for success
    acc = 0
    visited = []
    current = 0
    success = False
    while True:
        if current >= len(instructions_new):
            success = True
            break
        if current in visited:
            break
        visited.append(current)
        op = instructions_new[current]
        if op[0] == "acc":
            acc += op[1]
            current += 1
            continue
        if op[0] == "nop":
            current += 1
            continue
        if op[0] == "jmp":
            current += op[1]
            continue
    if success:
        break
print(acc)

In [ ]:
# Day 7
with open("input7", "r") as fp:
    lines = fp.readlines()
bags = {}
for line in lines:
    line = line.replace("bags", "").replace("bag", "").replace(".","")
    parts = line.split("contain")
    color = parts[0].strip()
    bags[color] = {}
    in_parts = parts[1].split(",")
    for part in in_parts:
        part = part.strip()
        if part == "no other":
            continue
        number = int(part.split()[0].strip())
        name = " ".join(part.split()[1:]).strip()
        bags[color][name] = number
def find_color(color):
    result = []
    for bag, in_bag in bags.items():
        if color in in_bag:
            result.append(bag)
            result.extend(find_color(bag))
    return result
print(len(np.unique(abc)))
def find_num(color):
    result = 1
    for bag, num in bags[color].items():
        result += find_num(bag)*num
    return result
print(find_num("shiny gold")-1)

In [ ]:
# Day 6
with open("input6", "r") as fp:
    lines = fp.readlines()
all_groups = []
current = []
for line in lines:
    if line.strip() == "":
        all_groups.append(current)
        current = []
    else:
        current.append(line.strip())
all_groups.append(current)
all_strings = ["".join(a) for a in all_groups]
all_unique = [len(np.unique(list(a))) for a in all_strings]
print(np.sum(all_unique))
all_every = []
for g in all_groups:
    if len(g) == 0:
        continue
    a = set(list(g[0]))
    if len(g) > 1:
        for b in g[1:]:
            a = a.intersection(set(list(b)))
    all_every.append(len(a))
print(np.sum(all_every))

In [ ]:
# Day 5
with open("input5", "r") as fp:
    lines = fp.readlines()
seat_nr = []
for line in lines:
    row = line[:7].replace("B","1").replace("F", "0")
    col = line[7:10].replace("R","1").replace("L", "0")
    row = int(row, base=2)
    col = int(col, base=2)
    id = row*8+col
    seat_nr.append([row, col, id])
print(np.max(np.array(seat_nr)[:,2]))
ids = [s[2] for s in seat_nr]
ids.sort()
diff = np.argmax(np.diff(ids))
print(np.sum(ids[diff:diff+2])//2)